## Imports

In [60]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
from tqdm import tqdm

# DL
from keras import layers

## ***Data Loading***

In [61]:
path = '../Datasets/COVID Qu Ex/Infection Segmentation Data'

In [62]:
dataset = keras.preprocessing.image_dataset_from_directory(
    directory=path,
    labels='inferred',
    color_mode='rgb',
    image_size=(120,120),
    shuffle=True,
    seed=50,
    batch_size=1
    
)

Found 5826 files belonging to 3 classes.


In [63]:
CLASS_NAMES = dataset.class_names
CLASS_NAMES

['COVID-19', 'Non-COVID', 'Normal']

In [64]:
type(dataset)

tensorflow.python.data.ops.batch_op._BatchDataset

In [65]:
data = []
for x, y in tqdm(dataset):
    data.append((x,y))
    

100%|██████████| 5826/5826 [00:02<00:00, 2176.10it/s]


In [66]:
df = pd.DataFrame(data, columns=['images','labels'])

In [67]:
def Tensor_toScalar(x):
    if isinstance(x, tf.Tensor):
        return x.numpy()
    else:
        return x

In [68]:
df.head()

,images,labels
0,"(((tf.Tensor([6.7811117 6.7811117 6.7811117], ...","(tf.Tensor(2, shape=(), dtype=int32))"
1,"(((tf.Tensor([13.245556 13.245556 13.245556], ...","(tf.Tensor(0, shape=(), dtype=int32))"
2,"(((tf.Tensor([39.545547 39.545547 39.545547], ...","(tf.Tensor(0, shape=(), dtype=int32))"
3,"(((tf.Tensor([22.133333 22.133333 22.133333], ...","(tf.Tensor(0, shape=(), dtype=int32))"
4,"(((tf.Tensor([0. 0. 0.], shape=(3,), dtype=flo...","(tf.Tensor(2, shape=(), dtype=int32))"


In [69]:
df = df.applymap(Tensor_toScalar)

In [70]:
df.head()

,images,labels
0,"[[[[6.7811117 6.7811117 6.7811117], [23.089998...",[2]
1,"[[[[13.245556 13.245556 13.245556], [13.696667...",[0]
2,"[[[[39.545547 39.545547 39.545547], [33.959995...",[0]
3,"[[[[22.133333 22.133333 22.133333], [22.133333...",[0]
4,"[[[[0. 0. 0.], [5.8966684 5.8966684 5.8966684]...",[2]


In [71]:
df['images'] = df['images']/255

In [72]:
df.head(2)

,images,labels
0,"[[[[0.0265926 0.0265926 0.0265926], [0.0905490...",[2]
1,"[[[[0.05194336 0.05194336 0.05194336], [0.0537...",[0]


## ***Exploratory Data Analisis***

### Data Modeling

In [73]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['images'],df['labels'],test_size=0.3)
x_test, x_val, y_test, y_val = train_test_split(x_test,y_test,test_size=0.5)

print(len(x_train))
print(len(x_test))
print(len(x_val))

4078
874
874


In [74]:
x_train = np.stack(x_train)
y_train = np.stack(y_train)
x_val = np.stack(x_val)
y_val = np.stack(y_val)
x_test = np.stack(x_test)
y_test = np.stack(y_test)

In [75]:
x_train = np.squeeze(x_train)
y_train = np.squeeze(y_train)
x_val = np.squeeze(x_val)
y_val = np.squeeze(y_val)
x_test = np.squeeze(x_test)
y_test = np.squeeze(y_test)

In [76]:
x_test.shape

(874, 120, 120, 3)

### Image Data Augmentation

In [77]:
keras.preprocessing.image.ImageDataGenerator

keras.src.preprocessing.image.ImageDataGenerator

In [78]:
def FeNet():
    
    model = keras.Sequential()

    model.add(layers.InputLayer((120,120,3)))
    model.add(layers.Conv2D(16,3,strides=2, padding='same'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(32,3,strides=2, padding='same'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))
    

    model.add(layers.SeparableConv2D(64,3,strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(3,activation='softmax'))


    model.compile(optimizer=keras.optimizers.Adam(0.0003),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [79]:
model = FeNet()
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 60, 60, 16)        448       
                                                                 
 max_pooling2d_27 (MaxPooli  (None, 30, 30, 16)        0         
 ng2D)                                                           
                                                                 
 batch_normalization_27 (Ba  (None, 30, 30, 16)        64        
 tchNormalization)                                               
                                                                 
 dropout_24 (Dropout)        (None, 30, 30, 16)        0         
                                                                 
 conv2d_40 (Conv2D)          (None, 15, 15, 32)        4640      
                                                                 
 max_pooling2d_28 (MaxPooli  (None, 7, 7, 32)        

In [80]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True)
lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, min_lr=0.00001,patience=3)

In [81]:
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=100, verbose=1, callbacks=[early_stop, lr])

Epoch 1/100
128/128 [==============================] - 8s 47ms/step - loss: 1.2174 - accuracy: 0.5135 - val_loss: 1.0686 - val_accuracy: 0.5126 - lr: 3.0000e-04
Epoch 2/100
128/128 [==============================] - 5s 35ms/step - loss: 0.8725 - accuracy: 0.6160 - val_loss: 1.0109 - val_accuracy: 0.5046 - lr: 3.0000e-04
Epoch 3/100
128/128 [==============================] - 5s 36ms/step - loss: 0.8457 - accuracy: 0.6425 - val_loss: 1.0044 - val_accuracy: 0.5618 - lr: 3.0000e-04
Epoch 4/100
128/128 [==============================] - 5s 36ms/step - loss: 0.8199 - accuracy: 0.6594 - val_loss: 0.8658 - val_accuracy: 0.6121 - lr: 3.0000e-04
Epoch 5/100
128/128 [==============================] - 5s 39ms/step - loss: 0.7981 - accuracy: 0.6724 - val_loss: 0.8422 - val_accuracy: 0.6236 - lr: 3.0000e-04
Epoch 6/100
128/128 [==============================] - 5s 37ms/step - loss: 0.7790 - accuracy: 0.6770 - val_loss: 0.7428 - val_accuracy: 0.6991 - lr: 3.0000e-04
Epoch 7/100
128/128 [=============